In [ ]:
!pip install selenium

In [ ]:
%%shell

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

apt-get update
apt-get install chromium chromium-driver

In [ ]:
from selenium import webdriver

In [ ]:
def web_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--verbose")
    options.add_argument('--no-sandbox')
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument("--window-size=1920, 1200")
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)
    return driver

In [ ]:
driver = web_driver()

In [ ]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

hal=11
berita=[]

for page in range(1,hal):
    driver.get(f"https://search.kompas.com/search/?q=Ganjar+Pranowo&submit=Submit#gsc.tab=0&gsc.q=Ganjar%20Pranowo&gsc.page={page}")
    wait = WebDriverWait(driver, 20)
    wait.until(EC.visibility_of_element_located(("css selector", 'a.gs-title')))
    time.sleep(1)
    list_berita= driver.find_elements("css selector", 'a.gs-title')
    for isi_list in list_berita:
       url_berita = isi_list.get_attribute('href')
       if url_berita != None and url_berita not in berita :
         berita.append(url_berita)
       else:
        continue


In [ ]:
berita

In [ ]:
import csv

driver = web_driver()
for isi in berita :
    driver.get(isi)
    try :
        judul_berita =driver.find_element("css selector",'h1.read__title').text
        tanggal_berita =driver.find_element("class name",'read__time').text
        query_isiBerita ='//div[@class="read__content"]/div[@class="clearfix"]'
        isi_berita =driver.find_elements("xpath", '{}/*[not(contains(@class, "inner-link-baca-juga"))]'.format(query_isiBerita))
        isi_bersih ="\n".join(x.text for x in isi_berita if not x.text.startswith("Baca juga"))

        with open('Ganjar.csv','a')as file:
            wr = csv.writer(file, delimiter=',')
            wr.writerow([judul_berita, tanggal_berita, isi_bersih])
            file.close()
    except :
        print("ISI ERROR : ",isi)

driver.quit()

ISI ERROR :  https://video.kompas.com/watch/599702/pemain-tim-sepak-bola-u-22-indonesia-kunjungi-rumah-ganjar-pranowo
ISI ERROR :  https://video.kompas.com/watch/597683/kisah-ganjar-pranowo-yang-pernah-jadi-tukang-ojek-di-purworejo
ISI ERROR :  https://video.kompas.com/watch/466609/nasdem-minta-ganjar-pranowo-bersyukur-ke-surya-paloh-usung-anies
ISI ERROR :  https://kolom.kompas.com/ganjar.pranowo
ISI ERROR :  https://www.kompas.com/tag/ganjar-pranowo


In [ ]:
import pandas as pd
import re
import csv
driver =web_driver()
isi = "https://nasional.kompas.com/read/2023/03/01/05500061/pilpres-2024-diprediksi-diikuti-3-capres--ganjar-prabowo-dan-anies"
driver.get(isi)
query_isiBerita ='//div[@class="read__content"]/div[@class="clearfix"]'
isi_berita =driver.find_elements("xpath", '{}/*[not(contains(@class, "inner-link-baca-juga"))]'.format(query_isiBerita))
print("\n".join(x.text for x in isi_berita if not x.text.startswith("Baca juga")))

driver.quit()


In [ ]:
import pandas as pd
data_pemilu =pd.read_csv('/content/Prabowo.csv',on_bad_lines='skip')
data_pemilu




In [ ]:
data_pemilu.to_excel(r'export_dataframe.xlsx', index=False)